In [ ]:
import os, sys

sys.path.append('..')

In [20]:
import pandas as pd
import argparse
from datetime import date,datetime
from datetime import timedelta
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from src.repositories.client_repository import ClientRepository

from utils import *
from dotenv import load_dotenv



In [ ]:
pd.set_option('display.max_columns', None)
load_dotenv()


## Get Client Information

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--client-id', type=int, required=True, help='Client ID')
# args = parser.parse_args()

# # Get environment variables
# client_id = args.client_id

In [ ]:

client_id = 2

engine = create_engine(os.environ['DATABASE_URI'])
session = sessionmaker(bind=engine)
client_repo = ClientRepository(session=session())
client = client_repo.fetch_client(client_id)
token = client.token
client_settings = client.settings

client_timezone_offset = int(client_settings['client_timezone_offset'])
category_name_col = str(client_settings['category_name_col'])

In [ ]:
client_settings

# Branches

In [ ]:
last_page = call_foodics('branches', 1, client_id, token, return_last_page=True)

list_responses = call_foodics('branches', last_page, client_id, token)



In [ ]:
df_branches = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_branches['client_id'] = client_id
df_branches['slug'] = df_branches.name.apply(lambda x: generate_slug(x))
df_branches

In [ ]:
df_branches = df_branches[['id', 'client_id', 'name', 'slug', 'opening_from', 'opening_to', 'created_at', 'updated_at', 'deleted_at']]

In [ ]:
df_branches

# Products

In [ ]:
last_page = call_foodics('products', 1, client_id,token, return_last_page=True)

list_responses = call_foodics('products', last_page, client_id,token, includables='category')

In [ ]:
df_products = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_products['client_id'] = client_id
df_products['slug'] = df_products.name.apply(lambda x: generate_slug(x))

In [ ]:
print(category_name_col)

df_products['category_name'] = df_products['category'].apply(lambda x: x[category_name_col])
df_products = df_products[~df_products.category_name.isna()]
df_products['category_name'] = df_products['category_name'].apply(lambda x: x.capitalize())

df_products['category_id'] = df_products['category'].apply(lambda x: x['id'])
df_products.rename(columns={'sku':"sku"}, inplace=True)
df_products.drop('category', axis=1, inplace=True)
# df_products = df_products[df_products.deleted_at.isna()]
df_products.head()

In [ ]:
df_products = df_products[['id', 'client_id', 'sku', 'slug', 'name', 'category_id', 'is_active', 'is_stock_product', 'price', 'created_at', 'updated_at', 'deleted_at']]

In [ ]:

df_products = df_products.rename(columns={'is_stock_product':"is_stock"})

In [ ]:
df_products.head()

# Categories

In [ ]:
last_page = call_foodics('categories', 1, client_id,token, return_last_page=True)


list_responses = call_foodics('categories', last_page, client_id, token,) 

In [ ]:
df_categories = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_categories['client_id'] = client_id
df_categories['slug'] = df_categories.name.apply(lambda x: generate_slug(x))
df_categories.head()

In [ ]:
cats_to_be_deleted = df_categories[~df_categories.deleted_at.isna()]
cats_to_be_deleted.head()

In [ ]:
df_categories = df_categories[df_categories.deleted_at.isna()]

In [ ]:
df_categories = df_categories[['id', 'client_id', category_name_col, 'slug', 'created_at', 'updated_at', 'deleted_at']].rename(columns={category_name_col:"name"})


In [ ]:
df_categories

In [ ]:
df_categories.name = df_categories.name.apply(lambda x: x.capitalize())

In [ ]:
print(df_categories['name'].tolist())

# Orders

In [ ]:
filter = {}
path = f'../../data/{client_id}/raw/orders_final_include.csv'


if os.path.exists(path):
    print('orders_final_include.csv already exists')
    df_orders = pd.read_csv(path)
    df_orders.reset_index(drop=True, inplace=True)  
else:
    print('orders_final_include.csv does not exist')
    includables = 'branch,products.product,products.options.modifier_option'
    # call the foodics api to get the orders
    last_page = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter, return_last_page=True)
    print('last_page: ', last_page)

    #temp
    last_page = 5
    
    list_responses = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter)
    df_orders = pd.DataFrame([item for sublist in list_responses for item in sublist])
    df_orders.to_csv(path, index=False)

# TODO: remove this line after testing
df_orders = pd.read_csv(path)
df_orders.reset_index(drop=True, inplace=True)
    

### Process Datetime columns

In [ ]:
df_orders.created_at = pd.to_datetime(df_orders.created_at)
max_date = df_orders.created_at.max().date().strftime('%Y-%m-%d')
max_date

In [ ]:
today = date.today().strftime('%Y-%m-%d')
today

In [ ]:
df_orders.head()

In [ ]:
df_orders['branch'] = df_orders['branch'].astype(str)

In [ ]:
df_orders['branch_id'] = df_orders['branch'].apply(lambda x: eval(x)['id'])
df_orders['client_id'] = client_id

In [ ]:
orders_header = df_orders[['id', 'client_id', 'branch_id',  'source', 'type', 'status', 'total_price', 'created_at', 'updated_at']]
orders_header.isnull().sum()

In [ ]:
orders_header.head()

In [ ]:
orders_header['ordered_at'] = orders_header['created_at']

# Order Details

In [ ]:
df_orders.head()

In [ ]:
# I want to create a dataframe with the following columns:
# order_id, product_id, category_id, quantity, price

# I will create a list of dictionaries, where each dictionary is a row in the dataframe
# I will then convert the list of dictionaries to a dataframe
import sys
import uuid

need_unavailable_category = False

list_order_details = []
length_orders = len(df_orders)
for index, row in df_orders.iterrows():

    order_header_id = row['id']
    branch_id = eval(row['branch'])['id']
    created_at = row['created_at']
    updated_at = row['updated_at']
    total_price = row['total_price']


    for order_product in eval(row['products']):
        order_details_id = str(uuid.uuid4())

        product_id = order_product['product']['id']
        try:
            category_id = df_products[df_products['id'] == product_id]['category_id'].values[0]
        except:
            need_unavailable_category = True
            category_id = '00000000-0000-0000-0000-000000000000'
        
        quantity = order_product['quantity']
        price = order_product['total_price']

        list_order_details.append({
                                   'id':order_details_id,
                                   'header_id':order_header_id,
                                   'product_id':product_id,
                                   'category_id':category_id,
                                   'client_id':client_id, 
                                   'quantity':quantity,
                                   'price':price,
                                   'created_at':created_at,
                                   'updated_at':updated_at})


In [ ]:
if need_unavailable_category:
    print("Yes, we need to add the unavailable category, make sure to add it in the .env")
    new_row = pd.DataFrame({'id': '00000000-0000-0000-0000-000000000000', 'name': 'Not Available', 'slug':"not-available", 'client_id':client_id}, index=[df_categories.index.max()+1])
    today = date.today().strftime('%Y-%m-%d')
    df_categories = pd.concat([df_categories, new_row], ignore_index=True)
    df_categories.created_at = df_categories.created_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.updated_at = df_categories.updated_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.deleted_at = df_categories.deleted_at.fillna(method='ffill')



df_categories

In [ ]:
orders_details = pd.DataFrame(list_order_details)

In [ ]:
orders_details.head()

In [ ]:
orders_details[orders_details.category_id.isna()].sort_values('created_at')

# Order Details Options

In [ ]:
df_options = pd.DataFrame(eval(df_orders.head()['products'][0])[0]['options'])
df_options.head()

#iterate over the df_orders dataframe and then iterate over the products column and then iterate over the options column
# and then iterate over the modifier_option column

list_order_options = []
for index, row in df_orders.iterrows():
    order_header_id = row['id']
    for order_product in eval(row['products']):
        product_id = order_product['product']['id']
        for order_option in order_product['options']:
            modifer_option = order_option['modifier_option']
            #get from modifier_option
            option_name = modifer_option['name']
            option_name_localized = modifer_option['name_localized']
            option_sku = modifer_option['sku']

            option_id = order_option['id']
            option_quantity = order_option['quantity']
            option_partition = order_option['partition']
            option_unit_price = order_option['unit_price']
            option_total_price = order_option['total_price']
            option_total_cost = order_option['total_cost']
        

            # get order_details_id by filtering df_order_details on product_id and order_header_id
            order_details_id = orders_details[(orders_details['product_id'] == product_id) & (orders_details['header_id'] == order_header_id)]['id'].values[0]

            list_order_options.append({
                'order_details_id': order_details_id,
                'option_id': option_id,
                'option_name': option_name,
                'option_name_localized': option_name_localized,
                'option_sku': option_sku,
                'option_quantity': option_quantity,
                'option_partition': option_partition,
                'option_unit_price': option_unit_price,
                'option_total_price': option_total_price,
                'option_total_cost': option_total_cost,
            })
                                        
df_options = pd.DataFrame(list_order_options)

In [ ]:
# filter df_options on order_details_id and option_id
df_orders[df_orders['id'] == '0002d85a-c450-4e76-9055-95ed0e8fbef0']['total_price']

In [ ]:
#grab an id from order_details
order_details_id = orders_details['id'].values[0]

df_options[df_options['order_details_id'] == order_details_id]

orders_details[orders_details['id'] == order_details_id]

In [ ]:
# There is no need to recalculate the order_details price by summing product price and options price. The products array comes with total price already calculated for each order details.

## KEEP THIS CODE FOR REFERENCE IF WE WANT TO MANUALLY CALCULATE THE ORDER_DETAILS PRICE##
# we need to adjust all order_details price coloumn, to be the sum of the column value and the sum of the df_options total_price column
# loop over the df_options dataframe and for each order_details_id, sum the total_price column and then update the orders_details dataframe

# for index, row in df_options.iterrows():
#     order_details_id = row['order_details_id']
#     total_price = row['option_total_price']
#     orders_details.loc[orders_details['id'] == order_details_id, 'price'] += total_price
    
# orders_details[orders_details['id'] == order_details_id]

## Change Data Timezone from UTC to Asia/Amm

In [ ]:
orders_header.head()

In [ ]:
orders_details.head()

In [ ]:
# import pytz

orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])

time_difference = timedelta(hours=client_timezone_offset)

orders_header['ordered_at'] += time_difference

In [ ]:
orders_header.head()

# Write to CSV

In [ ]:
df_products.to_csv(f'../../data/{client_id}/raw/products.csv', index=False)

In [ ]:
orders_details.to_csv(f'../../data/{client_id}/raw/order_details.csv', index=False)

In [ ]:
orders_header.to_csv(f'../../data/{client_id}/raw/order_header.csv', index=False)

In [ ]:
df_categories.to_csv(f'../../data/{client_id}/raw/categories.csv', index=False)

In [ ]:
df_branches.to_csv(f'../../data/{client_id}/raw/branches.csv', index=False)

In [ ]:
df_options.to_csv(f'../../data/{client_id}/raw/options.csv', index=False)

# Display all

In [ ]:
df_products.head()

In [ ]:
orders_details.head()

In [ ]:
orders_header.head()

In [ ]:
df_branches.head()

In [ ]:
df_categories.head()

In [ ]:
df_options.head()